# SECTION 1 - PRE PROCESSING

### Install Turicreate

In [0]:
!pip install turicreate

### Import necessary Libraries

In [0]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import turicreate as tc # Turicreate library for recommendations

### Google Drive Pre Requisite

In [0]:
# Code to read csv file into Colaboratory:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials 

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
link = 'https://drive.google.com/open?id=1dJ2MmNB9_UG3R5HZohSIqqu_dC0_N-S2'

fluff, id = link.split('=')

print (id) # Verify that you have everything after '='

### Read data from the file.

In [5]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile("steam-200k.csv")  
df = pd.read_csv("steam-200k.csv", header=None, index_col=None, names=['UserID', 'Game', 'Action', 'Hours', 'Other'])
df.head()

,UserID,Game,Action,Hours,Other
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0
2,151603712,Fallout 4,purchase,1.0,0
3,151603712,Fallout 4,play,87.0,0
4,151603712,Spore,purchase,1.0,0


### Check shape of the dataset

In [6]:
df.shape

(200000, 5)

### Check the datatypes of the columns

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 5 columns):
UserID    200000 non-null int64
Game      200000 non-null object
Action    200000 non-null object
Hours     200000 non-null float64
Other     200000 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 7.6+ MB


### Check the values in "Other" column

In [8]:
df["Other"].value_counts()

0    200000
Name: Other, dtype: int64

### Remove the column

In [0]:
df.drop(["Other"],axis=1,inplace = True)

### Statistical Information

In [10]:
df.loc[df["Action"]=="play","Hours"].describe()

count    70489.000000
mean        48.878063
std        229.335236
min          0.100000
25%          1.000000
50%          4.500000
75%         19.100000
max      11754.000000
Name: Hours, dtype: float64

# SECTION 2 - RANKING

### Dataset

In [11]:
df.head()

,UserID,Game,Action,Hours
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


### Groupby using Actions

In [12]:
df_group_values = df.groupby(["UserID","Action"]).size()
df_group_values.head(25)

UserID  Action  
5250    play          6
        purchase     21
76767   play         20
        purchase     36
86540   play         15
        purchase     82
103360  purchase     10
144736  play          1
        purchase      8
181212  play          2
        purchase     12
229911  play         18
        purchase     27
298950  play        175
        purchase    259
299153  purchase     14
381543  play          1
        purchase     10
547685  play          5
        purchase     25
554278  play          9
        purchase     28
561758  play         80
        purchase    148
577614  play          1
dtype: int64

### Describe the dataset

In [13]:
print("Number of unique games : {0}".format(len(df.Game.unique())))
print("Number of unique users : {0}".format(len(df.UserID.unique())))
print("Number of total purchases by users : {0}".format(len(df.loc[df['Action'] == 'purchase'])))
print("Number of total plays by users : {0}".format(len(df.loc[df['Action'] == 'play'])))

Number of unique games : 5155
Number of unique users : 12393
Number of total purchases by users : 129511
Number of total plays by users : 70489


### Create 2 new datasets 

In [0]:
df_purchase =  df.loc[df['Action'] == 'purchase', ['UserID','Game','Action','Hours']].rename(columns={'Hours': 'Purchased'})  # Purchased Games
df_play = df.loc[df['Action'] == 'play', ['UserID','Game','Hours']].rename(columns={'Hours': 'Played_hours'})                 # Played games by hour

In [15]:
print('shape of purchase dataset : ', df_purchase.shape[0])
print('shape of play dataset : ', df_play.shape[0])

shape of purchase dataset :  129511
shape of play dataset :  70489


### Merge datasets

In [0]:
total_dataset = pd.merge(df_purchase, df_play, left_on=['UserID','Game'], right_on=['UserID','Game'], how='left').fillna(0)

In [17]:
total_dataset.head(20)

,UserID,Game,Action,Purchased,Played_hours
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,273.0
1,151603712,Fallout 4,purchase,1.0,87.0
2,151603712,Spore,purchase,1.0,14.9
3,151603712,Fallout New Vegas,purchase,1.0,12.1
4,151603712,Left 4 Dead 2,purchase,1.0,8.9
5,151603712,HuniePop,purchase,1.0,8.5
6,151603712,Path of Exile,purchase,1.0,8.1
7,151603712,Poly Bridge,purchase,1.0,7.5
8,151603712,Left 4 Dead,purchase,1.0,3.3
9,151603712,Team Fortress 2,purchase,1.0,2.8


### Create a ranking system for games per user ID

In [0]:
#Ranking = lambda x: pd.qcut(x,10,labels=False,duplicates='drop')
#total_dataset['Ranking'] = total_dataset.groupby('UserID')['Played_hours'].apply(Ranking)

In [0]:
total_dataset['Ranking'] = total_dataset.groupby('UserID')['Played_hours'].transform(pd.cut, 10, labels=False,duplicates = "drop")+1.0

In [20]:
total_dataset.head(30)

,UserID,Game,Action,Purchased,Played_hours,Ranking
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,273.0,10.0
1,151603712,Fallout 4,purchase,1.0,87.0,4.0
2,151603712,Spore,purchase,1.0,14.9,1.0
3,151603712,Fallout New Vegas,purchase,1.0,12.1,1.0
4,151603712,Left 4 Dead 2,purchase,1.0,8.9,1.0
5,151603712,HuniePop,purchase,1.0,8.5,1.0
6,151603712,Path of Exile,purchase,1.0,8.1,1.0
7,151603712,Poly Bridge,purchase,1.0,7.5,1.0
8,151603712,Left 4 Dead,purchase,1.0,3.3,1.0
9,151603712,Team Fortress 2,purchase,1.0,2.8,1.0


In [21]:
total_dataset.loc[total_dataset["UserID"]==151603712].head(100)

,UserID,Game,Action,Purchased,Played_hours,Ranking
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,273.0,10.0
1,151603712,Fallout 4,purchase,1.0,87.0,4.0
2,151603712,Spore,purchase,1.0,14.9,1.0
3,151603712,Fallout New Vegas,purchase,1.0,12.1,1.0
4,151603712,Left 4 Dead 2,purchase,1.0,8.9,1.0
5,151603712,HuniePop,purchase,1.0,8.5,1.0
6,151603712,Path of Exile,purchase,1.0,8.1,1.0
7,151603712,Poly Bridge,purchase,1.0,7.5,1.0
8,151603712,Left 4 Dead,purchase,1.0,3.3,1.0
9,151603712,Team Fortress 2,purchase,1.0,2.8,1.0


### Create a clean dataset by dropping redundant columns

In [22]:
clean_df = total_dataset.drop(["Purchased","Action"],axis = 1)
clean_df.head(20)

,UserID,Game,Played_hours,Ranking
0,151603712,The Elder Scrolls V Skyrim,273.0,10.0
1,151603712,Fallout 4,87.0,4.0
2,151603712,Spore,14.9,1.0
3,151603712,Fallout New Vegas,12.1,1.0
4,151603712,Left 4 Dead 2,8.9,1.0
5,151603712,HuniePop,8.5,1.0
6,151603712,Path of Exile,8.1,1.0
7,151603712,Poly Bridge,7.5,1.0
8,151603712,Left 4 Dead,3.3,1.0
9,151603712,Team Fortress 2,2.8,1.0


### Display top 5 users based on number of games

In [23]:
clean_df["UserID"].value_counts()

62990992     1075
33865373      785
30246419      766
58345543      667
76892907      597
             ... 
149194171       1
207945140       1
130315685       1
282733934       1
214618086       1
Name: UserID, Length: 12393, dtype: int64

# SECTION 3 - RECOMMENDATION SYSTEM - FACTORIZATION RECOMMENDER

### Create a SFRame from the clean dataframe

In [0]:
dfS = tc.SFrame(clean_df)

### Factorization Recommender System


In [25]:
m = tc.factorization_recommender.create(dfS,
			                              user_id='UserID',
                                    item_id='Game',
                                    target='Ranking')

Preparing data set.

Data has 129535 observations with 12393 users and 5155 items.

Data prepared in: 0.169527s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 2.85355

Final training RMSE: 1.68924

In [26]:
recommendations = m.recommend()

recommendations finished on 1000/12393 queries. users per second: 5268.62

recommendations finished on 2000/12393 queries. users per second: 5314.61

recommendations finished on 3000/12393 queries. users per second: 5352.11

recommendations finished on 4000/12393 queries. users per second: 5367.46

recommendations finished on 5000/12393 queries. users per second: 5373.13

...

recommendations finished on 8000/12393 queries. users per second: 5360.84

recommendations finished on 9000/12393 queries. users per second: 5365.86

recommendations finished on 10000/12393 queries. users per second: 5378.74

recommendations finished on 11000/12393 queries. users per second: 5386.94

recommendations finished on 12000/12393 queries. users per second: 5440.16

### USER DESCRIPTION AND RECOMMENDATIONS

###TOP USER 1

In [27]:
fr1 = m.recommend(users=['151603712'])
fr1.head(20)

UserID,Game,score,rank
151603712,Counter-Strike,3.59921151535447,1
151603712,Counter-Strike Source,3.318337859081582,2
151603712,Counter-Strike GlobalOffensive ...,3.295660690831498,3
151603712,Half-Life 2,3.0173503472083367,4
151603712,Unturned,2.8074335767501153,5
151603712,Sid Meier's CivilizationV ...,2.735713945197419,6
151603712,Counter-Strike NexonZombies ...,2.7023700712793626,7
151603712,Call of Duty ModernWarfare 2 ...,2.666452520953969,8
151603712,Ricochet,2.642383040355996,9
151603712,Half-Life Blue Shift,2.63719497756894,10


### Creating a dataframe

In [28]:
fact1 = pd.DataFrame(fr1)
fact1.drop(["UserID","rank"],axis = 1, inplace = True)
fact1.rename(columns={'Game': 'Game_FR', 'score': 'score_FR'}, inplace=True)
fact1.head(10)

,Game_FR,score_FR
0,Counter-Strike,3.599212
1,Counter-Strike Source,3.318338
2,Counter-Strike Global Offensive,3.295661
3,Half-Life 2,3.017350
4,Unturned,2.807434
5,Sid Meier's Civilization V,2.735714
6,Counter-Strike Nexon Zombies,2.702370
7,Call of Duty Modern Warfare 2,2.666453
8,Ricochet,2.642383
9,Half-Life Blue Shift,2.637195


### Creating a dataframe

In [29]:
clean_df.loc[clean_df["UserID"] == 151603712,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
0,The Elder Scrolls V Skyrim,273.0,10.0
1,Fallout 4,87.0,4.0
2,Spore,14.9,1.0
3,Fallout New Vegas,12.1,1.0
4,Left 4 Dead 2,8.9,1.0
5,HuniePop,8.5,1.0
6,Path of Exile,8.1,1.0
7,Poly Bridge,7.5,1.0
8,Left 4 Dead,3.3,1.0
9,Team Fortress 2,2.8,1.0


###TOP USER 2

In [30]:
fr2 = m.recommend(users=['62990992'])
fr2

UserID,Game,score,rank
62990992,Unturned,2.781367094683007,1
62990992,Call of Duty ModernWarfare 2 ...,2.2532993866675652,2
62990992,H1Z1,2.13686308773096,3
62990992,Call of Duty ModernWarfare 2 - Multiplayer ...,2.1279049827330865,4
62990992,Call of Duty Black Ops,2.1200462146275796,5
62990992,Grand Theft Auto V,2.0452180965417184,6
62990992,The Sims(TM) 3,2.042751071470336,7
62990992,Football Manager 2009,2.03585664691265,8
62990992,Train Simulator,1.9970836150073565,9
62990992,Farming Simulator 2013,1.9390092915468968,10


### Creating a dataframe

In [31]:
fact2 = pd.DataFrame(fr2)
fact2.drop(["UserID","rank"],axis = 1, inplace = True)
fact2.rename(columns={'Game': 'Game_FR', 'score': 'score_FR'}, inplace=True)
fact2.head(10)

,Game_FR,score_FR
0,Unturned,2.781367
1,Call of Duty Modern Warfare 2,2.253299
2,H1Z1,2.136863
3,Call of Duty Modern Warfare 2 - Multiplayer,2.127905
4,Call of Duty Black Ops,2.120046
5,Grand Theft Auto V,2.045218
6,The Sims(TM) 3,2.042751
7,Football Manager 2009,2.035857
8,Train Simulator,1.997084
9,Farming Simulator 2013,1.939009


In [32]:
clean_df.loc[clean_df["UserID"] == 62990992,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
77076,Counter-Strike Global Offensive,663.0,10.0
77077,Sid Meier's Civilization V,550.0,9.0
77078,Total War SHOGUN 2,212.0,4.0
77079,Total War ROME II - Emperor Edition,198.0,3.0
77080,Dungeon Defenders,195.0,3.0
...,...,...,...
78146,Xenophage,0.0,1.0
78147,Xpand Rally Xtreme,0.0,1.0
78148,Zen Bound 2,0.0,1.0
78149,Zombie Driver HD Apocalypse Pack,0.0,1.0


###TOP USER 3

In [33]:
fr3 = m.recommend(users=['33865373'])
fr3

UserID,Game,score,rank
33865373,Counter-Strike Source,3.306293712424592,1
33865373,Counter-Strike GlobalOffensive ...,3.1577509357207574,2
33865373,Unturned,2.7964980615847863,3
33865373,Sid Meier's CivilizationV ...,2.6391002400391854,4
33865373,Call of Duty ModernWarfare 2 - Multiplayer ...,2.522734777497605,5
33865373,Counter-Strike NexonZombies ...,2.492555038499192,6
33865373,Grand Theft Auto V,2.42958857254918,7
33865373,Trove,2.4283634586804665,8
33865373,H1Z1,2.423694336282567,9
33865373,The Sims(TM) 3,2.4022689326279916,10


### Creating a dataframe

In [34]:
fact3 = pd.DataFrame(fr3)
fact3.drop(["UserID","rank"],axis = 1, inplace = True)
fact3.rename(columns={'Game': 'Game_FR', 'score': 'score_FR'}, inplace=True)
fact3.head(10)

,Game_FR,score_FR
0,Counter-Strike Source,3.306294
1,Counter-Strike Global Offensive,3.157751
2,Unturned,2.796498
3,Sid Meier's Civilization V,2.639100
4,Call of Duty Modern Warfare 2 - Multiplayer,2.522735
5,Counter-Strike Nexon Zombies,2.492555
6,Grand Theft Auto V,2.429589
7,Trove,2.428363
8,H1Z1,2.423694
9,The Sims(TM) 3,2.402269


In [35]:
clean_df.loc[clean_df["UserID"] == 33865373,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
47739,The Elder Scrolls V Skyrim,345.0,10.0
47740,The Elder Scrolls IV Oblivion,239.0,7.0
47741,Fallout New Vegas,198.0,6.0
47742,Sid Meier's Civilization IV,135.0,4.0
47743,Sid Meier's Civilization IV,2.0,1.0
...,...,...,...
48519,Xenophage,0.0,1.0
48520,Xotic,0.0,1.0
48521,Zafehouse Diaries,0.0,1.0
48522,Zen Bound 2,0.0,1.0


###TOP USER 4

In [36]:
fr4 = m.recommend(users=['30246419'])
fr4

UserID,Game,score,rank
30246419,Dota 2,5.824189187454538,1
30246419,Counter-Strike Source,4.697782964872674,2
30246419,Counter-Strike,4.690150575208024,3
30246419,Team Fortress 2,4.636831962930517,4
30246419,Counter-Strike GlobalOffensive ...,4.632633538412408,5
30246419,Call of Duty ModernWarfare 2 - Multiplayer ...,4.030859352516488,6
30246419,Grand Theft Auto V,3.975383253263787,7
30246419,Garry's Mod,3.874509127306298,8
30246419,Counter-Strike NexonZombies ...,3.8483098237388886,9
30246419,Trove,3.842202009128884,10


### Creating a dataframe

In [37]:
fact4 = pd.DataFrame(fr4)
fact4.drop(["UserID","rank"],axis = 1, inplace = True)
fact4.rename(columns={'Game': 'Game_FR', 'score': 'score_FR'}, inplace=True)
fact4.head(10)

,Game_FR,score_FR
0,Dota 2,5.824189
1,Counter-Strike Source,4.697783
2,Counter-Strike,4.690151
3,Team Fortress 2,4.636832
4,Counter-Strike Global Offensive,4.632634
5,Call of Duty Modern Warfare 2 - Multiplayer,4.030859
6,Grand Theft Auto V,3.975383
7,Garry's Mod,3.874509
8,Counter-Strike Nexon Zombies,3.848310
9,Trove,3.842202


In [38]:
clean_df.loc[clean_df["UserID"] == 30246419,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
62101,The Witcher 3 Wild Hunt,99.0,10.0
62102,Fallout 4,97.0,10.0
62103,Two Worlds II,61.0,7.0
62104,Test Drive Unlimited 2,57.0,6.0
62105,Middle-earth Shadow of Mordor,53.0,6.0
...,...,...,...
62862,Yury,0.0,1.0
62863,Z,0.0,1.0
62864,Zafehouse Diaries,0.0,1.0
62865,Zeno Clash 2,0.0,1.0


###TOP USER 5

In [39]:
fr5 = m.recommend(users=['58345543'])
fr5

UserID,Game,score,rank
58345543,Counter-Strike,5.290025697516755,1
58345543,Grand Theft Auto V,4.757214368748025,2
58345543,Euro Truck Simulator 2,4.65266645090516,3
58345543,ARK Survival Evolved,4.442864121364908,4
58345543,H1Z1,4.428638325499849,5
58345543,The Sims(TM) 3,4.41312779622014,6
58345543,Empire Total War,4.35109972344573,7
58345543,Football Manager 2009,4.305280254649476,8
58345543,Cities Skylines,4.30124661462243,9
58345543,Farming Simulator 15,4.28386157886918,10


### Creating a dataframe

In [40]:
fact5 = pd.DataFrame(fr5)
fact5.drop(["UserID","rank"],axis = 1, inplace = True)
fact5.rename(columns={'Game': 'Game_FR', 'score': 'score_FR'}, inplace=True)
fact5.head(10)

,Game_FR,score_FR
0,Counter-Strike,5.290026
1,Grand Theft Auto V,4.757214
2,Euro Truck Simulator 2,4.652666
3,ARK Survival Evolved,4.442864
4,H1Z1,4.428638
5,The Sims(TM) 3,4.413128
6,Empire Total War,4.351100
7,Football Manager 2009,4.305280
8,Cities Skylines,4.301247
9,Farming Simulator 15,4.283862


In [41]:
clean_df.loc[clean_df["UserID"] == 58345543,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
115598,Spiral Knights,70.0,10.0
115599,Blacklight Retribution,64.0,10.0
115600,Dungeon Defenders,56.0,8.0
115601,Terraria,54.0,8.0
115602,Borderlands 2,53.0,8.0
...,...,...,...
116260,Wrath of Athena,0.0,1.0
116261,WTFast Gamers Private Network (GPN),0.0,1.0
116262,Xam,0.0,1.0
116263,You Have to Win the Game,0.0,1.0


In [42]:
clean_df.loc[clean_df["UserID"] == 58345543,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
115598,Spiral Knights,70.0,10.0
115599,Blacklight Retribution,64.0,10.0
115600,Dungeon Defenders,56.0,8.0
115601,Terraria,54.0,8.0
115602,Borderlands 2,53.0,8.0
...,...,...,...
116260,Wrath of Athena,0.0,1.0
116261,WTFast Gamers Private Network (GPN),0.0,1.0
116262,Xam,0.0,1.0
116263,You Have to Win the Game,0.0,1.0


# SECTION 4 - RECOMMENDATION SYSTEM - RANKING FACTORIZATION RECOMMENDER

### Ranking Factorization Recommender System

In [43]:
n = tc.ranking_factorization_recommender.create(dfS,
			                              user_id='UserID',
                                    item_id='Game',
                                    target='Ranking')


Preparing data set.

Data has 129535 observations with 12393 users and 5155 items.

Data prepared in: 0.164534s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 6.63614

Final training RMSE: 1.90184

In [44]:
recommendations = n.recommend()

recommendations finished on 1000/12393 queries. users per second: 2512.76

recommendations finished on 2000/12393 queries. users per second: 2582.7

recommendations finished on 3000/12393 queries. users per second: 2600.68

recommendations finished on 4000/12393 queries. users per second: 2591.34

recommendations finished on 5000/12393 queries. users per second: 2612.77

...

recommendations finished on 8000/12393 queries. users per second: 2611.95

recommendations finished on 9000/12393 queries. users per second: 2619.69

recommendations finished on 10000/12393 queries. users per second: 2623.94

recommendations finished on 11000/12393 queries. users per second: 2627.2

recommendations finished on 12000/12393 queries. users per second: 2619.03

### USER DESCRIPTION AND RECOMMENDATIONS

###TOP USER 1

In [45]:
rf1 = n.recommend(users=['151603712'])
rf1.head(20)

UserID,Game,score,rank
151603712,Counter-Strike,3.042427854108171,1
151603712,Counter-Strike Source,2.97718675570424,2
151603712,Counter-Strike GlobalOffensive ...,2.9645570232146543,3
151603712,Unturned,2.918730975555734,4
151603712,Sid Meier's CivilizationV ...,2.5077969147437376,5
151603712,Half-Life 2,2.130364136266069,6
151603712,Terraria,1.9603886468880933,7
151603712,Portal 2,1.9563434286587995,8
151603712,Call of Duty ModernWarfare 2 - Multiplayer ...,1.9504931731694501,9
151603712,Call of Duty ModernWarfare 2 ...,1.8722515119784635,10


###Creating a dataframe

In [46]:
ranfact1 = pd.DataFrame(rf1)
ranfact1.drop(["UserID","rank"],axis = 1, inplace = True)
ranfact1.rename(columns={'Game': 'Game_RFR', 'score': 'score_RFR'}, inplace=True)
ranfact1.head(10)

,Game_RFR,score_RFR
0,Counter-Strike,3.042428
1,Counter-Strike Source,2.977187
2,Counter-Strike Global Offensive,2.964557
3,Unturned,2.918731
4,Sid Meier's Civilization V,2.507797
5,Half-Life 2,2.130364
6,Terraria,1.960389
7,Portal 2,1.956343
8,Call of Duty Modern Warfare 2 - Multiplayer,1.950493
9,Call of Duty Modern Warfare 2,1.872252


In [47]:
clean_df.loc[clean_df["UserID"] == 151603712,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
0,The Elder Scrolls V Skyrim,273.0,10.0
1,Fallout 4,87.0,4.0
2,Spore,14.9,1.0
3,Fallout New Vegas,12.1,1.0
4,Left 4 Dead 2,8.9,1.0
5,HuniePop,8.5,1.0
6,Path of Exile,8.1,1.0
7,Poly Bridge,7.5,1.0
8,Left 4 Dead,3.3,1.0
9,Team Fortress 2,2.8,1.0


###TOP USER 2

In [48]:
rf2 = n.recommend(users=['62990992'])
rf2

UserID,Game,score,rank
62990992,Unturned,1.9923604442351621,1
62990992,Call of Duty ModernWarfare 2 - Multiplayer ...,0.9596423520320219,2
62990992,Call of Duty ModernWarfare 2 ...,0.9409424974196714,3
62990992,PAYDAY 2,0.9305881573432249,4
62990992,Call of Duty Black Ops -Multiplayer ...,0.7651392533057493,5
62990992,Grand Theft Auto V,0.7189708365672391,6
62990992,DayZ,0.7025823904746336,7
62990992,Don't Starve,0.6945310248606962,8
62990992,Fallout New Vegas,0.6890297545665067,9
62990992,Dark Souls Prepare to DieEdition ...,0.6585998995774549,10


###Creating a dataframe

In [49]:
ranfact2 = pd.DataFrame(rf2)
ranfact2.drop(["UserID","rank"],axis = 1, inplace = True)
ranfact2.rename(columns={'Game': 'Game_RFR', 'score': 'score_RFR'}, inplace=True)
ranfact2.head(10)

,Game_RFR,score_RFR
0,Unturned,1.992360
1,Call of Duty Modern Warfare 2 - Multiplayer,0.959642
2,Call of Duty Modern Warfare 2,0.940942
3,PAYDAY 2,0.930588
4,Call of Duty Black Ops - Multiplayer,0.765139
5,Grand Theft Auto V,0.718971
6,DayZ,0.702582
7,Don't Starve,0.694531
8,Fallout New Vegas,0.689030
9,Dark Souls Prepare to Die Edition,0.658600


In [50]:
clean_df.loc[clean_df["UserID"] == 62990992,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
77076,Counter-Strike Global Offensive,663.0,10.0
77077,Sid Meier's Civilization V,550.0,9.0
77078,Total War SHOGUN 2,212.0,4.0
77079,Total War ROME II - Emperor Edition,198.0,3.0
77080,Dungeon Defenders,195.0,3.0
...,...,...,...
78146,Xenophage,0.0,1.0
78147,Xpand Rally Xtreme,0.0,1.0
78148,Zen Bound 2,0.0,1.0
78149,Zombie Driver HD Apocalypse Pack,0.0,1.0


###TOP USER 3

In [51]:
rf3 = n.recommend(users=['33865373'])
rf3

UserID,Game,score,rank
33865373,Counter-Strike GlobalOffensive ...,2.660006792711572,1
33865373,Counter-Strike Source,2.535124482082681,2
33865373,Unturned,2.324752600359277,3
33865373,Sid Meier's CivilizationV ...,2.093989954757051,4
33865373,Call of Duty ModernWarfare 2 - Multiplayer ...,1.4246517135375303,5
33865373,Call of Duty ModernWarfare 2 ...,1.3406461192839902,6
33865373,PAYDAY 2,1.29155799882348,7
33865373,Warframe,1.2790919257872861,8
33865373,War Thunder,1.2046479745381635,9
33865373,Age of Empires II HDEdition ...,1.2004426135295194,10


###Creating a dataframe

In [52]:
ranfact3 = pd.DataFrame(rf3)
ranfact3.drop(["UserID","rank"],axis = 1, inplace = True)
ranfact3.rename(columns={'Game': 'Game_RFR', 'score': 'score_RFR'}, inplace=True)
ranfact3.head(10)

,Game_RFR,score_RFR
0,Counter-Strike Global Offensive,2.660007
1,Counter-Strike Source,2.535124
2,Unturned,2.324753
3,Sid Meier's Civilization V,2.093990
4,Call of Duty Modern Warfare 2 - Multiplayer,1.424652
5,Call of Duty Modern Warfare 2,1.340646
6,PAYDAY 2,1.291558
7,Warframe,1.279092
8,War Thunder,1.204648
9,Age of Empires II HD Edition,1.200443


In [53]:
clean_df.loc[clean_df["UserID"] == 33865373,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
47739,The Elder Scrolls V Skyrim,345.0,10.0
47740,The Elder Scrolls IV Oblivion,239.0,7.0
47741,Fallout New Vegas,198.0,6.0
47742,Sid Meier's Civilization IV,135.0,4.0
47743,Sid Meier's Civilization IV,2.0,1.0
...,...,...,...
48519,Xenophage,0.0,1.0
48520,Xotic,0.0,1.0
48521,Zafehouse Diaries,0.0,1.0
48522,Zen Bound 2,0.0,1.0


###TOP USER 4

In [54]:
rf4 = n.recommend(users=['30246419'])
rf4

UserID,Game,score,rank
30246419,Dota 2,4.965038002895669,1
30246419,Team Fortress 2,3.9061815305226606,2
30246419,Counter-Strike GlobalOffensive ...,3.458225251602487,3
30246419,Counter-Strike Source,3.422756434845285,4
30246419,Counter-Strike,3.09822446243699,5
30246419,Unturned,2.661734135317163,6
30246419,Garry's Mod,2.564247698950128,7
30246419,Call of Duty ModernWarfare 2 - Multiplayer ...,2.3955105854743284,8
30246419,Counter-Strike ConditionZero ...,2.2976321889632505,9
30246419,Grand Theft Auto V,2.1702982498877805,10


###Creating a dataframe

In [55]:
ranfact4 = pd.DataFrame(rf4)
ranfact4.drop(["UserID","rank"],axis = 1, inplace = True)
ranfact4.rename(columns={'Game': 'Game_RFR', 'score': 'score_RFR'}, inplace=True)
ranfact4.head(10)

,Game_RFR,score_RFR
0,Dota 2,4.965038
1,Team Fortress 2,3.906182
2,Counter-Strike Global Offensive,3.458225
3,Counter-Strike Source,3.422756
4,Counter-Strike,3.098224
5,Unturned,2.661734
6,Garry's Mod,2.564248
7,Call of Duty Modern Warfare 2 - Multiplayer,2.395511
8,Counter-Strike Condition Zero,2.297632
9,Grand Theft Auto V,2.170298


In [56]:
clean_df.loc[clean_df["UserID"] == 30246419,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
62101,The Witcher 3 Wild Hunt,99.0,10.0
62102,Fallout 4,97.0,10.0
62103,Two Worlds II,61.0,7.0
62104,Test Drive Unlimited 2,57.0,6.0
62105,Middle-earth Shadow of Mordor,53.0,6.0
...,...,...,...
62862,Yury,0.0,1.0
62863,Z,0.0,1.0
62864,Zafehouse Diaries,0.0,1.0
62865,Zeno Clash 2,0.0,1.0


###TOP USER 5

In [57]:
rf5 = n.recommend(users=['58345543'])
rf5

UserID,Game,score,rank
58345543,Counter-Strike,3.84277573363717,1
58345543,Grand Theft Auto V,2.809548230337457,2
58345543,Call of Duty Black Ops,2.77978819744523,3
58345543,Empire Total War,2.724582613872842,4
58345543,Euro Truck Simulator 2,2.6393348290198606,5
58345543,Age of Empires II HDEdition ...,2.62063634531434,6
58345543,Arma 2 OperationArrowhead ...,2.5727702094786924,7
58345543,Rust,2.571987630295114,8
58345543,ARK Survival Evolved,2.562704445289926,9
58345543,Call of Duty ModernWarfare 3 ...,2.559012831615762,10


###Creating a dataframe


In [58]:
ranfact5 = pd.DataFrame(rf5)
ranfact5.drop(["UserID","rank"],axis = 1, inplace = True)
ranfact5.rename(columns={'Game': 'Game_RFR', 'score': 'score_RFR'}, inplace=True)
ranfact5.head(10)

,Game_RFR,score_RFR
0,Counter-Strike,3.842776
1,Grand Theft Auto V,2.809548
2,Call of Duty Black Ops,2.779788
3,Empire Total War,2.724583
4,Euro Truck Simulator 2,2.639335
5,Age of Empires II HD Edition,2.620636
6,Arma 2 Operation Arrowhead,2.572770
7,Rust,2.571988
8,ARK Survival Evolved,2.562704
9,Call of Duty Modern Warfare 3,2.559013


In [59]:
clean_df.loc[clean_df["UserID"] == 58345543,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
115598,Spiral Knights,70.0,10.0
115599,Blacklight Retribution,64.0,10.0
115600,Dungeon Defenders,56.0,8.0
115601,Terraria,54.0,8.0
115602,Borderlands 2,53.0,8.0
...,...,...,...
116260,Wrath of Athena,0.0,1.0
116261,WTFast Gamers Private Network (GPN),0.0,1.0
116262,Xam,0.0,1.0
116263,You Have to Win the Game,0.0,1.0


# SECTION 5 - RECOMMENDATION SYSTEM - ITEM SIMILARITY RECOMMENDER

### Item Similarity Recommender System

In [60]:
o = tc.item_similarity_recommender.create(dfS,
			                              user_id='UserID',
                                    item_id='Game',
                                    target='Ranking')

Warning: Ignoring columns Played_hours;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 129535 observations with 12393 users and 5155 items.

Data prepared in: 0.135447s

...

| 543.377ms                           | 100              | 5155            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.566492s

In [61]:
recommendations = o.recommend()

recommendations finished on 1000/12393 queries. users per second: 17044.5

recommendations finished on 2000/12393 queries. users per second: 18045.5

recommendations finished on 3000/12393 queries. users per second: 18460.1

recommendations finished on 4000/12393 queries. users per second: 18568.3

recommendations finished on 5000/12393 queries. users per second: 18192.4

...

recommendations finished on 8000/12393 queries. users per second: 18369.5

recommendations finished on 9000/12393 queries. users per second: 18320.6

recommendations finished on 10000/12393 queries. users per second: 18206.2

recommendations finished on 11000/12393 queries. users per second: 18110.5

recommendations finished on 12000/12393 queries. users per second: 18179.2

### USER DESCRIPTION AND RECOMMENDATIONS

###TOP 1 USER

In [62]:
isr1 = o.recommend(users=['151603712'])
isr1.head(20)

UserID,Game,score,rank
151603712,Portal 2,0.10231157541275024,1
151603712,Borderlands 2,0.10229341983795166,2
151603712,The Witcher 2 Assassinsof Kings Enhanced Edi ...,0.10084794908761978,3
151603712,Dishonored,0.09983683824539184,4
151603712,Metro 2033,0.09556290954351425,5
151603712,Batman Arkham City GOTY,0.09495677947998046,6
151603712,Saints Row The Third,0.08762047588825225,7
151603712,Skyrim High ResolutionTexture Pack ...,0.08500545769929886,8
151603712,Deus Ex Human Revolution,0.08309174180030823,9
151603712,Batman Arkham Asylum GOTYEdition ...,0.08188620805740357,10


###Creating a dataframe

In [63]:
itsim1 = pd.DataFrame(isr1)
itsim1.drop(["UserID","rank"],axis = 1, inplace = True)
itsim1.rename(columns={'Game': 'Game_ISR', 'score': 'score_ISR'}, inplace=True)
itsim1.head(10)

,Game_ISR,score_ISR
0,Portal 2,0.102312
1,Borderlands 2,0.102293
2,The Witcher 2 Assassins of Kings Enhanced Edition,0.100848
3,Dishonored,0.099837
4,Metro 2033,0.095563
5,Batman Arkham City GOTY,0.094957
6,Saints Row The Third,0.087620
7,Skyrim High Resolution Texture Pack,0.085005
8,Deus Ex Human Revolution,0.083092
9,Batman Arkham Asylum GOTY Edition,0.081886


In [64]:
clean_df.loc[clean_df["UserID"] == 151603712,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
0,The Elder Scrolls V Skyrim,273.0,10.0
1,Fallout 4,87.0,4.0
2,Spore,14.9,1.0
3,Fallout New Vegas,12.1,1.0
4,Left 4 Dead 2,8.9,1.0
5,HuniePop,8.5,1.0
6,Path of Exile,8.1,1.0
7,Poly Bridge,7.5,1.0
8,Left 4 Dead,3.3,1.0
9,Team Fortress 2,2.8,1.0


###TOP 2 USER

In [65]:
isr2 = o.recommend(users=['62990992'])
isr2

UserID,Game,score,rank
62990992,Overlord Raising Hell,0.021720042277811172,1
62990992,Brtal Legend,0.02107142313812556,2
62990992,Mark of the Ninja,0.019560436575153793,3
62990992,Bastion,0.01940129328979535,4
62990992,Darksiders,0.017975371540262457,5
62990992,Dishonored,0.015522174620896244,6
62990992,LIMBO,0.014491602499386792,7
62990992,Rise of the Argonauts,0.01429952619674054,8
62990992,The Walking Dead,0.014220608754104443,9
62990992,Cave Story+,0.013507157954830356,10


###Creating a dataframe

In [66]:
itsim2 = pd.DataFrame(isr2)
itsim2.drop(["UserID","rank"],axis = 1, inplace = True)
itsim2.rename(columns = {'Game': 'Game_ISR', 'score': 'score_ISR'},inplace = True)
itsim2.head(10)

,Game_ISR,score_ISR
0,Overlord Raising Hell,0.021720
1,Brtal Legend,0.021071
2,Mark of the Ninja,0.019560
3,Bastion,0.019401
4,Darksiders,0.017975
5,Dishonored,0.015522
6,LIMBO,0.014492
7,Rise of the Argonauts,0.014300
8,The Walking Dead,0.014221
9,Cave Story+,0.013507


In [67]:
clean_df.loc[clean_df["UserID"] == 62990992,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
77076,Counter-Strike Global Offensive,663.0,10.0
77077,Sid Meier's Civilization V,550.0,9.0
77078,Total War SHOGUN 2,212.0,4.0
77079,Total War ROME II - Emperor Edition,198.0,3.0
77080,Dungeon Defenders,195.0,3.0
...,...,...,...
78146,Xenophage,0.0,1.0
78147,Xpand Rally Xtreme,0.0,1.0
78148,Zen Bound 2,0.0,1.0
78149,Zombie Driver HD Apocalypse Pack,0.0,1.0


###TOP 3 USER

In [68]:
isr3 = o.recommend(users=['33865373'])
isr3

UserID,Game,score,rank
33865373,GoD Factory Wingmen,0.021196801407435507,1
33865373,Guacamelee! Gold Edition,0.021057773112636253,2
33865373,The Swapper,0.019111334202215842,3
33865373,Titan Quest,0.019020690131433232,4
33865373,The Walking Dead,0.017936440011889664,5
33865373,The Witcher 2 Assassinsof Kings Enhanced Edi ...,0.01760121419564965,6
33865373,Gray Matter,0.017418147823245255,7
33865373,Orcs Must Die! 2,0.017022938688391262,8
33865373,Overlord Raising Hell,0.01663894052665258,9
33865373,Borderlands 2,0.015909244719239855,10


###Creating a dataframe

In [69]:
itsim3 = pd.DataFrame(isr3)
itsim3.drop(["UserID","rank"],axis = 1, inplace = True)
itsim3.rename(columns={'Game': 'Game_ISR', 'score': 'score_ISR'}, inplace=True)
itsim3.head(10)

,Game_ISR,score_ISR
0,GoD Factory Wingmen,0.021197
1,Guacamelee! Gold Edition,0.021058
2,The Swapper,0.019111
3,Titan Quest,0.019021
4,The Walking Dead,0.017936
5,The Witcher 2 Assassins of Kings Enhanced Edition,0.017601
6,Gray Matter,0.017418
7,Orcs Must Die! 2,0.017023
8,Overlord Raising Hell,0.016639
9,Borderlands 2,0.015909


In [70]:
clean_df.loc[clean_df["UserID"] == 33865373,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
47739,The Elder Scrolls V Skyrim,345.0,10.0
47740,The Elder Scrolls IV Oblivion,239.0,7.0
47741,Fallout New Vegas,198.0,6.0
47742,Sid Meier's Civilization IV,135.0,4.0
47743,Sid Meier's Civilization IV,2.0,1.0
...,...,...,...
48519,Xenophage,0.0,1.0
48520,Xotic,0.0,1.0
48521,Zafehouse Diaries,0.0,1.0
48522,Zen Bound 2,0.0,1.0


###TOP 4 USER

In [71]:
isr4 = o.recommend(users=['30246419'])
isr4

UserID,Game,score,rank
30246419,Sine Mora,0.023019409069744808,1
30246419,Dishonored,0.02031866167530869,2
30246419,Dear Esther,0.02001262512131642,3
30246419,BioShock Infinite,0.019627176804347786,4
30246419,Intrusion 2,0.01695912825103177,5
30246419,Osmos,0.01676778493348475,6
30246419,Gratuitous Space Battles,0.01669403985240871,7
30246419,Red Faction GuerrillaSteam Edition ...,0.016295533170813156,8
30246419,Proteus,0.016212027534665798,9
30246419,Thomas Was Alone,0.015154094642644186,10


###Creating a dataframe

In [72]:
itsim4 = pd.DataFrame(isr4)
itsim4.drop(["UserID","rank"],axis = 1, inplace = True)
itsim4.rename(columns={'Game': 'Game_ISR', 'score': 'score_ISR'}, inplace=True)
itsim4.head(10)

,Game_ISR,score_ISR
0,Sine Mora,0.023019
1,Dishonored,0.020319
2,Dear Esther,0.020013
3,BioShock Infinite,0.019627
4,Intrusion 2,0.016959
5,Osmos,0.016768
6,Gratuitous Space Battles,0.016694
7,Red Faction Guerrilla Steam Edition,0.016296
8,Proteus,0.016212
9,Thomas Was Alone,0.015154


In [73]:
clean_df.loc[clean_df["UserID"] == 30246419,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
62101,The Witcher 3 Wild Hunt,99.0,10.0
62102,Fallout 4,97.0,10.0
62103,Two Worlds II,61.0,7.0
62104,Test Drive Unlimited 2,57.0,6.0
62105,Middle-earth Shadow of Mordor,53.0,6.0
...,...,...,...
62862,Yury,0.0,1.0
62863,Z,0.0,1.0
62864,Zafehouse Diaries,0.0,1.0
62865,Zeno Clash 2,0.0,1.0


###TOP 5 USER

In [74]:
isr5 = o.recommend(users=['58345543'])
isr5

UserID,Game,score,rank
58345543,Combat Arms,0.02556089524392251,1
58345543,Dragon Nest Europe,0.02290467644954945,2
58345543,Global Agenda,0.02009778230397909,3
58345543,Dear Esther,0.015767073130106425,4
58345543,Frozen Synapse,0.0157638413233084,5
58345543,Mark of the Ninja,0.015663843165646802,6
58345543,Revenge of the Titans,0.015100526111619966,7
58345543,Dishonored,0.01461870391089637,8
58345543,FTL Faster Than Light,0.014549173302836605,9
58345543,Batman Arkham City GOTY,0.014514459235532148,10


###Creating a dataframe

In [75]:
itsim5 = pd.DataFrame(isr5)
itsim5.drop(["UserID","rank"],axis = 1, inplace = True)
itsim5.rename(columns={'Game': 'Game_ISR', 'score': 'score_ISR'},inplace = True)
itsim5.head(10)

,Game_ISR,score_ISR
0,Combat Arms,0.025561
1,Dragon Nest Europe,0.022905
2,Global Agenda,0.020098
3,Dear Esther,0.015767
4,Frozen Synapse,0.015764
5,Mark of the Ninja,0.015664
6,Revenge of the Titans,0.015101
7,Dishonored,0.014619
8,FTL Faster Than Light,0.014549
9,Batman Arkham City GOTY,0.014514


In [76]:
clean_df.loc[clean_df["UserID"] == 58345543,["Game","Played_hours","Ranking"]]

,Game,Played_hours,Ranking
115598,Spiral Knights,70.0,10.0
115599,Blacklight Retribution,64.0,10.0
115600,Dungeon Defenders,56.0,8.0
115601,Terraria,54.0,8.0
115602,Borderlands 2,53.0,8.0
...,...,...,...
116260,Wrath of Athena,0.0,1.0
116261,WTFast Gamers Private Network (GPN),0.0,1.0
116262,Xam,0.0,1.0
116263,You Have to Win the Game,0.0,1.0


### TOP 1 USER TOTAL COMPARISON FOR RECOMMENDATIONS

In [77]:
Top1_comparison = pd.concat([fact1, ranfact1,itsim1], axis=1, sort=False)
Top1_comparison.head(10)

,Game_FR,score_FR,Game_RFR,score_RFR,Game_ISR,score_ISR
0,Counter-Strike,3.599212,Counter-Strike,3.042428,Portal 2,0.102312
1,Counter-Strike Source,3.318338,Counter-Strike Source,2.977187,Borderlands 2,0.102293
2,Counter-Strike Global Offensive,3.295661,Counter-Strike Global Offensive,2.964557,The Witcher 2 Assassins of Kings Enhanced Edition,0.100848
3,Half-Life 2,3.017350,Unturned,2.918731,Dishonored,0.099837
4,Unturned,2.807434,Sid Meier's Civilization V,2.507797,Metro 2033,0.095563
5,Sid Meier's Civilization V,2.735714,Half-Life 2,2.130364,Batman Arkham City GOTY,0.094957
6,Counter-Strike Nexon Zombies,2.702370,Terraria,1.960389,Saints Row The Third,0.087620
7,Call of Duty Modern Warfare 2,2.666453,Portal 2,1.956343,Skyrim High Resolution Texture Pack,0.085005
8,Ricochet,2.642383,Call of Duty Modern Warfare 2 - Multiplayer,1.950493,Deus Ex Human Revolution,0.083092
9,Half-Life Blue Shift,2.637195,Call of Duty Modern Warfare 2,1.872252,Batman Arkham Asylum GOTY Edition,0.081886


### TOP 2 USER TOTAL COMPARISON FOR RECOMMENDATIONS

In [78]:
Top2_comparison = pd.concat([fact2, ranfact2,itsim2], axis=1, sort=False)
Top2_comparison.head(10)

,Game_FR,score_FR,Game_RFR,score_RFR,Game_ISR,score_ISR
0,Unturned,2.781367,Unturned,1.992360,Overlord Raising Hell,0.021720
1,Call of Duty Modern Warfare 2,2.253299,Call of Duty Modern Warfare 2 - Multiplayer,0.959642,Brtal Legend,0.021071
2,H1Z1,2.136863,Call of Duty Modern Warfare 2,0.940942,Mark of the Ninja,0.019560
3,Call of Duty Modern Warfare 2 - Multiplayer,2.127905,PAYDAY 2,0.930588,Bastion,0.019401
4,Call of Duty Black Ops,2.120046,Call of Duty Black Ops - Multiplayer,0.765139,Darksiders,0.017975
5,Grand Theft Auto V,2.045218,Grand Theft Auto V,0.718971,Dishonored,0.015522
6,The Sims(TM) 3,2.042751,DayZ,0.702582,LIMBO,0.014492
7,Football Manager 2009,2.035857,Don't Starve,0.694531,Rise of the Argonauts,0.014300
8,Train Simulator,1.997084,Fallout New Vegas,0.689030,The Walking Dead,0.014221
9,Farming Simulator 2013,1.939009,Dark Souls Prepare to Die Edition,0.658600,Cave Story+,0.013507


### TOP 3 USER TOTAL COMPARISON FOR RECOMMENDATIONS

In [79]:
Top3_comparison = pd.concat([fact3, ranfact3,itsim3], axis=1, sort=False)
Top3_comparison.head(10)

,Game_FR,score_FR,Game_RFR,score_RFR,Game_ISR,score_ISR
0,Counter-Strike Source,3.306294,Counter-Strike Global Offensive,2.660007,GoD Factory Wingmen,0.021197
1,Counter-Strike Global Offensive,3.157751,Counter-Strike Source,2.535124,Guacamelee! Gold Edition,0.021058
2,Unturned,2.796498,Unturned,2.324753,The Swapper,0.019111
3,Sid Meier's Civilization V,2.639100,Sid Meier's Civilization V,2.093990,Titan Quest,0.019021
4,Call of Duty Modern Warfare 2 - Multiplayer,2.522735,Call of Duty Modern Warfare 2 - Multiplayer,1.424652,The Walking Dead,0.017936
5,Counter-Strike Nexon Zombies,2.492555,Call of Duty Modern Warfare 2,1.340646,The Witcher 2 Assassins of Kings Enhanced Edition,0.017601
6,Grand Theft Auto V,2.429589,PAYDAY 2,1.291558,Gray Matter,0.017418
7,Trove,2.428363,Warframe,1.279092,Orcs Must Die! 2,0.017023
8,H1Z1,2.423694,War Thunder,1.204648,Overlord Raising Hell,0.016639
9,The Sims(TM) 3,2.402269,Age of Empires II HD Edition,1.200443,Borderlands 2,0.015909


### TOP 4 USER TOTAL COMPARISON FOR RECOMMENDATIONS

In [80]:
Top4_comparison = pd.concat([fact4, ranfact4,itsim4], axis=1, sort=False)
Top4_comparison.head(10)

,Game_FR,score_FR,Game_RFR,score_RFR,Game_ISR,score_ISR
0,Dota 2,5.824189,Dota 2,4.965038,Sine Mora,0.023019
1,Counter-Strike Source,4.697783,Team Fortress 2,3.906182,Dishonored,0.020319
2,Counter-Strike,4.690151,Counter-Strike Global Offensive,3.458225,Dear Esther,0.020013
3,Team Fortress 2,4.636832,Counter-Strike Source,3.422756,BioShock Infinite,0.019627
4,Counter-Strike Global Offensive,4.632634,Counter-Strike,3.098224,Intrusion 2,0.016959
5,Call of Duty Modern Warfare 2 - Multiplayer,4.030859,Unturned,2.661734,Osmos,0.016768
6,Grand Theft Auto V,3.975383,Garry's Mod,2.564248,Gratuitous Space Battles,0.016694
7,Garry's Mod,3.874509,Call of Duty Modern Warfare 2 - Multiplayer,2.395511,Red Faction Guerrilla Steam Edition,0.016296
8,Counter-Strike Nexon Zombies,3.848310,Counter-Strike Condition Zero,2.297632,Proteus,0.016212
9,Trove,3.842202,Grand Theft Auto V,2.170298,Thomas Was Alone,0.015154


### TOP 5 USER TOTAL COMPARISON FOR RECOMMENDATIONS

In [81]:
Top5_comparison = pd.concat([fact5, ranfact5,itsim5], axis=1, sort=False)
Top5_comparison.head(10)

,Game_FR,score_FR,Game_RFR,score_RFR,Game_ISR,score_ISR
0,Counter-Strike,5.290026,Counter-Strike,3.842776,Combat Arms,0.025561
1,Grand Theft Auto V,4.757214,Grand Theft Auto V,2.809548,Dragon Nest Europe,0.022905
2,Euro Truck Simulator 2,4.652666,Call of Duty Black Ops,2.779788,Global Agenda,0.020098
3,ARK Survival Evolved,4.442864,Empire Total War,2.724583,Dear Esther,0.015767
4,H1Z1,4.428638,Euro Truck Simulator 2,2.639335,Frozen Synapse,0.015764
5,The Sims(TM) 3,4.413128,Age of Empires II HD Edition,2.620636,Mark of the Ninja,0.015664
6,Empire Total War,4.351100,Arma 2 Operation Arrowhead,2.572770,Revenge of the Titans,0.015101
7,Football Manager 2009,4.305280,Rust,2.571988,Dishonored,0.014619
8,Cities Skylines,4.301247,ARK Survival Evolved,2.562704,FTL Faster Than Light,0.014549
9,Farming Simulator 15,4.283862,Call of Duty Modern Warfare 3,2.559013,Batman Arkham City GOTY,0.014514
